In [18]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
from scipy import stats

# Data preperation and exploration

## 1. Calculate total building load
        total_load(kW) = main_meter(kW) + PV_battery_system(kW)  

**There shouldn't have negative values in total_load(kW)**.  

Nigative values are rounded to 0.

## 2. Visualize total_load(kW) to understand the patterns of the data
Pay attention to weekly and seasonal patterns for different building types: office, school, nursing home, shopping center, kindergarten.

## 3. Merge the load csv files with corresponding weather csv files
All weather files are in 1 hour resolution. When combined with building load files at different time resolution, interpolation of weather files can be considered.

In [130]:
# Calculate building electricity total load (total electricity consumption)
def calculate_total_load(input_path, output_path):
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # List all CSV files in the input directory
    csv_files = [f for f in os.listdir(input_path) if f.endswith('.csv')]

    for file in csv_files:
        # Read the CSV file
        df = pd.read_csv(os.path.join(input_path, file), parse_dates=['timestamp'])

        # Check if the building has PV and battery system
        has_pv_battery = 'PV_battery_system(kW)' in df.columns

        # Calculate total load
        if has_pv_battery:
            df['total_load(kW)'] = df['main_meter(kW)'] + df['PV_battery_system(kW)']
        else:
            df['total_load(kW)'] = df['main_meter(kW)']

        # Select only timestamp and total_load columns
        df_output = df[['timestamp', 'total_load(kW)']]

        # Write to output CSV file
        output_file = os.path.join(output_path, file.replace('_uncleaned', '_simple_cleaned'))
        df_output.to_csv(output_file, index=False)

        print(f"Processed {file}")

    print("All files processed successfully.")

# Usage
input_path = "explore/01_load_uncleaned/"
output_path = "explore/02_load_simple_cleaned/"
calculate_total_load(input_path, output_path)

Processed L10.B01_1H.csv
Processed L09.B01_15min.csv
Processed L14.B02_1H.csv
Processed L09.B01_1H.csv
Processed L10.B01_15min.csv
Processed L06.B01_1H.csv
Processed L14.B04_1H.csv
Processed L14.B01_1H.csv
Processed L14.B03_1H.csv
Processed L09.B01_30min.csv
Processed L09.B01_5min.csv
Processed L10.B01_30min.csv
Processed L03.B02_1H.csv
Processed L14.B05_1H.csv
All files processed successfully.


In [12]:
# Check the data quality
def check_raw_load(file_path):
    print(f"Checking file: {os.path.basename(file_path)}")
    
    # Read the CSV file
    df = pd.read_csv(file_path, parse_dates=['timestamp'])
    
    # 1. Time Format Consistency
    if df['timestamp'].dtype != 'datetime64[ns, UTC]':
        print(f"Warning: Timestamp datatype is {df['timestamp'].dtype}, expected datetime64[ns, UTC]")
    
    # 2. Time Zone Consideration
    if not all(ts.tzinfo == df['timestamp'].iloc[0].tzinfo for ts in df['timestamp']):
        print("Warning: Inconsistent time zones found in timestamps")
    
    # 3. Duplicated Timestamps
    duplicates = df[df.duplicated('timestamp', keep=False)]
    if not duplicates.empty:
        print("Duplicated timestamps found:")
        print(duplicates)
    
    # 4. Missing Timestamps
    df = df.set_index('timestamp').sort_index()
    expected_freq = df.index.to_series().diff().mode().iloc[0]
    expected_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=expected_freq)
    missing_timestamps = expected_range.difference(df.index)
    if not missing_timestamps.empty:
        print("Missing timestamps:")
        print(missing_timestamps)
    
    # 5. Non-numeric Values in total_load(kW)
    non_numeric = df[pd.to_numeric(df['total_load(kW)'], errors='coerce').isna()]
    if not non_numeric.empty:
        print("Non-numeric values found in total_load(kW):")
        print(non_numeric)
    
    print("Check completed.\n")

def check_all_load_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            check_raw_load(file_path)

# Usage
directory = "train_public/load_simple_cleaned/"
check_all_load_files(directory)

Checking file: L10.B01_1H.csv
Missing timestamps:
DatetimeIndex(['2020-06-24 01:00:00+00:00', '2020-06-24 02:00:00+00:00',
               '2020-06-24 03:00:00+00:00', '2020-06-24 04:00:00+00:00',
               '2020-06-24 05:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='h')
Non-numeric values found in total_load(kW):
                           total_load(kW)
timestamp                                
2020-07-02 01:00:00+00:00             NaN
Check completed.

Checking file: L09.B01_15min.csv
Missing timestamps:
DatetimeIndex(['2021-01-08 11:45:00+00:00', '2021-01-08 12:00:00+00:00',
               '2021-01-08 12:15:00+00:00', '2021-01-08 12:30:00+00:00',
               '2021-01-08 12:45:00+00:00', '2021-01-08 13:00:00+00:00',
               '2021-01-08 13:15:00+00:00', '2021-01-08 13:30:00+00:00',
               '2021-01-08 13:45:00+00:00', '2021-01-08 14:00:00+00:00',
               ...
               '2021-10-18 03:15:00+00:00', '2021-10-18 03:30:00+00:00',
        

In [19]:
# Warning! This step is abandoned

# Fill the missing timestamps

def fill_missing_timestamps(input_file, output_dir):
    # Read the CSV file
    df = pd.read_csv(input_file, parse_dates=['timestamp'])
    
    # Set timestamp as index
    df.set_index('timestamp', inplace=True)
    
    # Determine the time resolution
    time_diff = df.index.to_series().diff().mode().iloc[0]
    
    # Create a complete time range
    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=time_diff)
    
    # Reindex the dataframe to include all timestamps
    df = df.reindex(full_range)
    
    # Convert non-numeric values to np.nan
    df['total_load(kW)'] = pd.to_numeric(df['total_load(kW)'], errors='coerce')
    
    # Reset index to make timestamp a column again
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'timestamp'}, inplace=True)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Save the processed dataframe
    output_file = os.path.join(output_dir, os.path.basename(input_file))
    df.to_csv(output_file, index=False)
    
    print(f"Processed {os.path.basename(input_file)}")

def process_all_files(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        if filename.endswith('.csv'):
            input_file = os.path.join(input_dir, filename)
            fill_missing_timestamps(input_file, output_dir)
    
    print("All files processed successfully.")

# Usage
input_dir = "explore/02_load_simple_cleaned/"
output_dir = "explore/03_load_timestamp_filled/"
process_all_files(input_dir, output_dir)


Processed L10.B01_1H.csv
Processed L09.B01_15min.csv
Processed L14.B02_1H.csv
Processed L09.B01_1H.csv
Processed L10.B01_15min.csv
Processed L06.B01_1H.csv
Processed L14.B04_1H.csv
Processed L14.B01_1H.csv
Processed L14.B03_1H.csv
Processed L09.B01_30min.csv
Processed L09.B01_5min.csv
Processed L10.B01_30min.csv
Processed L03.B02_1H.csv
Processed L14.B05_1H.csv
All files processed successfully.


In [131]:
# Visualize building load

def visualize_building_load(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path, parse_dates=['timestamp'])
    
    # Calculate time ranges
    start_time = df['timestamp'].min()
    end_time = df['timestamp'].max()
    time_delta = (end_time - start_time) / 4
    
    # Set up the plot
    fig, axs = plt.subplots(4, 1, figsize=(20, 20))
    fig.suptitle(f'Building Load - {os.path.basename(file_path)}', fontsize=16)
    
    for i in range(4):
        ax = axs[i]
        range_start = start_time + i * time_delta
        range_end = start_time + (i + 1) * time_delta
        
        # Filter data for the current time range
        mask = (df['timestamp'] >= range_start) & (df['timestamp'] < range_end)
        sub_df = df[mask]
        
        # Plot non-missing values
        non_missing_mask = sub_df['total_load(kW)'].notna()
        ax.plot(sub_df.loc[non_missing_mask, 'timestamp'], 
                sub_df.loc[non_missing_mask, 'total_load(kW)'], 
                label='Total Load')
        
        # Mark missing values
        missing_mask = sub_df['total_load(kW)'].isna()
        if missing_mask.any():
            ax.scatter(sub_df.loc[missing_mask, 'timestamp'], 
                       [ax.get_ylim()[0]] * missing_mask.sum(),  # Place at bottom of plot
                       color='red', label='Missing Values', zorder=5)
        
        # Identify and mark potential outliers
        non_null_data = sub_df.loc[non_missing_mask, 'total_load(kW)']
        if len(non_null_data) > 1:  # Need at least 2 points to calculate z-score
            z_scores = np.abs(stats.zscore(non_null_data))
            outliers_mask = z_scores > 3
            outlier_indices = non_null_data.index[outliers_mask]
            
            ax.scatter(sub_df.loc[outlier_indices, 'timestamp'], 
                       sub_df.loc[outlier_indices, 'total_load(kW)'], 
                       color='orange', marker='s', s=50, label='Potential Outliers', zorder=10)
        
        # Set labels
        ax.set_xlabel('Timestamp')
        ax.set_ylabel('Total Load (kW)')
        
        # Add legend
        ax.legend()
        
        # Rotate x-axis labels for better readability
        ax.tick_params(axis='x', rotation=45)
        
        # Set title for each subplot
        ax.set_title(f'Part {i+1}: {range_start} to {range_end}')
    
    # Adjust layout and save the plot
    plt.tight_layout()
    output_path = os.path.join(os.path.dirname(file_path), f"{os.path.splitext(os.path.basename(file_path))[0]}_plot.png")
    plt.savefig(output_path)
    plt.close()
    
    print(f"Plot saved: {output_path}")

def visualize_all_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            visualize_building_load(file_path)
    
    print("All files processed successfully.")

# Usage
directory = "explore/02_load_simple_cleaned/"
visualize_all_files(directory)

Plot saved: explore/02_load_simple_cleaned/L10.B01_1H_plot.png
Plot saved: explore/02_load_simple_cleaned/L09.B01_15min_plot.png
Plot saved: explore/02_load_simple_cleaned/L14.B02_1H_plot.png
Plot saved: explore/02_load_simple_cleaned/L09.B01_1H_plot.png
Plot saved: explore/02_load_simple_cleaned/L10.B01_15min_plot.png
Plot saved: explore/02_load_simple_cleaned/L06.B01_1H_plot.png
Plot saved: explore/02_load_simple_cleaned/L14.B04_1H_plot.png
Plot saved: explore/02_load_simple_cleaned/L14.B01_1H_plot.png
Plot saved: explore/02_load_simple_cleaned/L14.B03_1H_plot.png
Plot saved: explore/02_load_simple_cleaned/L09.B01_30min_plot.png
Plot saved: explore/02_load_simple_cleaned/L09.B01_5min_plot.png
Plot saved: explore/02_load_simple_cleaned/L10.B01_30min_plot.png
Plot saved: explore/02_load_simple_cleaned/L03.B02_1H_plot.png
Plot saved: explore/02_load_simple_cleaned/L14.B05_1H_plot.png
All files processed successfully.


In [136]:
# Check the shape of the cleaned files:

directory = 'explore/02_load_simple_cleaned/'
for filename in os.listdir(directory):
    if '.csv' in filename:
        filepath = os.path.join(directory, filename)
        print(filepath)

explore/02_load_simple_cleaned/L10.B01_1H.csv
explore/02_load_simple_cleaned/L09.B01_15min.csv
explore/02_load_simple_cleaned/L14.B02_1H.csv
explore/02_load_simple_cleaned/L09.B01_1H.csv
explore/02_load_simple_cleaned/L10.B01_15min.csv
explore/02_load_simple_cleaned/L06.B01_1H.csv
explore/02_load_simple_cleaned/L14.B04_1H.csv
explore/02_load_simple_cleaned/L14.B01_1H.csv
explore/02_load_simple_cleaned/L14.B03_1H.csv
explore/02_load_simple_cleaned/L09.B01_30min.csv
explore/02_load_simple_cleaned/L09.B01_5min.csv
explore/02_load_simple_cleaned/L10.B01_30min.csv
explore/02_load_simple_cleaned/L03.B02_1H.csv
explore/02_load_simple_cleaned/L14.B05_1H.csv


In [143]:
# Merge building load files with weather files

# read a CSV file and converts the timestamp column to a datetime index.
def read_and_preprocess_csv(file_path, timestamp_col='timestamp'):
    df = pd.read_csv(file_path)
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    df.set_index(timestamp_col, inplace=True)
    return df

# resamples the weather data to the target frequency using time-based interpolation.
def resample_weather_data(weather_df, target_freq):
    # Resample weather data to the target frequency
    return weather_df.resample(target_freq).interpolate(method='time')

# Reads and preprocesses both the building load and weather CSV files.
# Infers the frequency of the building load data.
# Resamples the weather data to match the building load data frequency.
# Merges the datasets based on the timestamp index.
# Saves the merged data to a new CSV file.
def merge_building_and_weather(building_file, weather_file, output_file):
    # Read and preprocess the CSV files
    building_df = read_and_preprocess_csv(building_file)
    weather_df = read_and_preprocess_csv(weather_file)

    # Determine the frequency of the building load data
    if '5min' in os.path.basename(building_file): 
        # Resample weather data to match building load data frequency
        resampled_weather_df = resample_weather_data(weather_df, '5min')
    elif '15min' in os.path.basename(building_file):
        # Resample weather data to match building load data frequency
        resampled_weather_df = resample_weather_data(weather_df, '15min')
    elif '30min' in os.path.basename(building_file):
        # Resample weather data to match building load data frequency
        resampled_weather_df = resample_weather_data(weather_df, '30min')
    else:
        resampled_weather_df = resample_weather_data(weather_df, '1h')

    # Merge the datasets
    merged_df = pd.merge(building_df, resampled_weather_df, left_index=True, right_index=True, how='left')
    print(merged_df.shape)

    # Save the merged data to a new CSV file
    merged_df.to_csv(output_file)
    print(f"Merged data saved to {output_file}")

def process_all_files(building_load_dir, weather_dir, output_dir):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Process each building load file
    for building_file in os.listdir(building_load_dir):
        if building_file.endswith('.csv'):
            building_path = os.path.join(building_load_dir, building_file)
            location = building_file.split('.')[0]
            weather_file = f"{location}_weather_train.csv"  # Assuming weather files are named with a "weather_" prefix
            weather_path = os.path.join(weather_dir, weather_file)
            output_file = os.path.join(output_dir, f"merged_{building_file}")

            if os.path.exists(weather_path):
                merge_building_and_weather(building_path, weather_path, output_file)
            else:
                print(f"Weather file not found for {building_file}")

# Usage example
building_load_dir = 'explore/02_load_simple_cleaned/'
weather_dir = 'explore/weather/'
output_dir = 'explore/04_weather_combined/'

process_all_files(building_load_dir, weather_dir, output_dir)

(13867, 7)
Merged data saved to explore/04_weather_combined/merged_L10.B01_1H.csv
(33044, 7)
Merged data saved to explore/04_weather_combined/merged_L09.B01_15min.csv
(8760, 7)
Merged data saved to explore/04_weather_combined/merged_L14.B02_1H.csv
(8287, 7)
Merged data saved to explore/04_weather_combined/merged_L09.B01_1H.csv
(55460, 7)
Merged data saved to explore/04_weather_combined/merged_L10.B01_15min.csv
(7674, 7)
Merged data saved to explore/04_weather_combined/merged_L06.B01_1H.csv
(8760, 7)
Merged data saved to explore/04_weather_combined/merged_L14.B04_1H.csv
(8760, 7)
Merged data saved to explore/04_weather_combined/merged_L14.B01_1H.csv
(8760, 7)
Merged data saved to explore/04_weather_combined/merged_L14.B03_1H.csv
(16554, 7)
Merged data saved to explore/04_weather_combined/merged_L09.B01_30min.csv
(98978, 7)
Merged data saved to explore/04_weather_combined/merged_L09.B01_5min.csv
(27731, 7)
Merged data saved to explore/04_weather_combined/merged_L10.B01_30min.csv
(8400, 7

# Data cleaning

Drop rows with **missing values** (alternative: filling missing values with mean/median or other imputation techniques). No need to fill them becasue there are no recordings to validate.    

Handling **outliers** (using e.x. IQR method etc.). But perhaps it is better choose not to handle the "outliers" as these may represent extreme weather conditions, so they are actually not outliers.  

Handling **improper values**. Values in the columns of "total_load(kW)", "direct_solar_radiation(W/m^2)" and "diffuse_solar_radiation(W/m^2)" should not be negative. All negative values in these columns should be rounded to 0.  

**Data type conversion**: Ensure all columns have appropriate data types. "timestamp" in datatime, others in numerical values.  

**Feature creation** and handling cyclical features: Create additional time-based features such as hour, day_of_week etc. and use sine/cosine transformation.

Data **normalisation**: Normalise or standerdise usually comes at the last step of data cleaning. The values to make sure they are in the same scale.  


In [150]:
# Data cleaning

from sklearn.preprocessing import StandardScaler
from pathlib import Path

def clean_and_preprocess_data(input_file_path, output_folder="explore/05_cleaned/"):
    # Load the data
    df = pd.read_csv(input_file_path)
    
    print(f"Original data shape: {df.shape}")

    
    # 1. Fill missing values with 0
    df.fillna(0, inplace=True)
    print(f"Shape after dropping missing values: {df.shape}")
    
    
    # 2. Handling improper values
    columns_to_clean = ["total_load(kW)", "direct_solar_radiation(W/m^2)", "diffuse_solar_radiation(W/m^2)"]
    for col in columns_to_clean:
        df[col] = np.maximum(df[col], 0)
    
    # 3. Data type conversion
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    numeric_columns = ['air_temperature_at_2m(deg_C)', 'relative_humidity_at_2m(%)', 
                       'direct_solar_radiation(W/m^2)', 'diffuse_solar_radiation(W/m^2)', 
                       'wind_speed_at_10m(km/h)', 'wind_direction_at_10m(deg)', 'total_load(kW)']
    df[numeric_columns] = df[numeric_columns].astype(float)
    
    # 4. Feature Creation
    df['hour'] = df['timestamp'].dt.hour
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    df['month'] = df['timestamp'].dt.month
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    
    # 5. Handling Cyclical Features
    def cyclical_encode(data, column, max_val):
        data[f'{column}_sin'] = np.sin(2 * np.pi * data[column]/max_val)
        data[f'{column}_cos'] = np.cos(2 * np.pi * data[column]/max_val)
        return data

    df = cyclical_encode(df, 'hour', 24)
    df = cyclical_encode(df, 'day_of_week', 7)
    df = cyclical_encode(df, 'month', 12)
    df = cyclical_encode(df, 'wind_direction_at_10m(deg)', 360)
    
    # Drop original cyclical columns
    df = df.drop(['hour', 'day_of_week', 'month', 'wind_direction_at_10m(deg)'], axis=1)
    
    # 6. Check for Multicollinearity
    correlation_matrix = df.corr()
    print("\nCorrelation Matrix:")
    print(correlation_matrix)
    
    # Identify highly correlated features
    high_corr_threshold = 0.9
    high_corr_features = np.where(np.abs(correlation_matrix) > high_corr_threshold)
    high_corr_features = [(correlation_matrix.index[x], correlation_matrix.columns[y]) 
                          for x, y in zip(*high_corr_features) if x != y and x < y]
    if high_corr_features:
        print("\nHighly correlated features:")
        for feat1, feat2 in high_corr_features:
            print(f"{feat1} - {feat2}: {correlation_matrix.loc[feat1, feat2]:.2f}")
    else:
        print("\nNo highly correlated features found.")
    
    # 7. Data Integrity Checks
    print("\nData Integrity Checks:")
    print(df.isnull().sum())
    
    for column in df.columns:
        if df[column].dtype in ['int64', 'float64']:
            print(f"{column}: Min = {df[column].min()}, Max = {df[column].max()}")
    
    # 8. Save Preprocessed Data
    output_path = Path(output_folder)
    output_path.mkdir(parents=True, exist_ok=True)
    output_file = output_path / Path(input_file_path).name.replace(".csv", "_cleaned.csv")
    df.to_csv(output_file, index=False)
    print(f"\nCleaned data saved to: {output_file}")
    
    return df

# Usage
input_folder = "explore/04_weather_combined"
for filename in os.listdir(input_folder):
    if filename.endswith(".csv"):
        input_file_path = os.path.join(input_folder, filename)
        print(f"\nProcessing file: {filename}")
        cleaned_df = clean_and_preprocess_data(input_file_path)


Processing file: merged_L03.B02_1H.csv
Original data shape: (8400, 8)
Shape after dropping missing values: (8400, 8)

Correlation Matrix:
                                timestamp  total_load(kW)  \
timestamp                        1.000000       -0.103311   
total_load(kW)                  -0.103311        1.000000   
air_temperature_at_2m(deg_C)     0.675074       -0.323806   
relative_humidity_at_2m(%)      -0.145625        0.107667   
direct_solar_radiation(W/m^2)    0.113233       -0.074006   
diffuse_solar_radiation(W/m^2)   0.145981        0.020935   
wind_speed_at_10m(km/h)         -0.060901        0.042697   
is_weekend                       0.009390       -0.460305   
hour_sin                        -0.002215        0.190634   
hour_cos                        -0.000292       -0.230592   
day_of_week_sin                 -0.014533        0.383639   
day_of_week_cos                  0.006999       -0.052741   
month_sin                       -0.765612        0.171598   
month_c

In [151]:
# Random forest
import csv
from pathlib import Path

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

def analyze_feature_importance(input_folder="explore/05_cleaned/"):
    for filename in os.listdir(input_folder):
        if filename.endswith("_cleaned.csv"):
            file_path = os.path.join(input_folder, filename)
            print(f"\nAnalyzing file: {filename}")
            
            # Load the CSV file
            df = pd.read_csv(file_path)
            
            # Separate features and target
            X = df.drop(['timestamp', 'total_load(kW)'], axis=1)
            y = df['total_load(kW)']
            
            # Standardize the features
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
                        # Random Forest for Feature Importance
            rf = RandomForestRegressor(n_estimators=100, random_state=42)
            rf.fit(X_scaled, y)
            
            # Get feature importances
            importances = rf.feature_importances_
            feature_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
            feature_importance = feature_importance.sort_values('importance', ascending=False)
            
            # Visualize Feature Importances
            plt.figure(figsize=(12, 8))
            sns.barplot(x='importance', y='feature', data=feature_importance)
            plt.title(f'Feature Importances from Random Forest - {filename}')
            plt.tight_layout()
            plt.savefig(f'feature_importances_{filename[:-4]}.png')
            plt.close()
            
            print("\nTop 10 Important Features:")
            print(feature_importance.head(10))

            # Create a list to store results
            results = []
            
            # Populate the results list
            for i, (feature, importance) in enumerate(zip(feature_importance['feature'], feature_importance['importance']), 1):
                results.append([filename, i, feature, importance])
            
            # Create the output directory if it doesn't exist
            output_dir = Path('explore/06_analysis/')
            output_dir.mkdir(exist_ok=True)
            
            # Write results to CSV
            output_file = output_dir / f'random_forest_{filename[:-4]}.csv'
            with open(output_file, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(['filename', 'feature_number', 
                                 'feature_name', 'importance'])
                writer.writerows(results)
            
            print(f"Feature importance results saved to {output_file}")



# Run the analysis
analyze_feature_importance()


Analyzing file: merged_L14.B04_1H_cleaned.csv

Top 10 Important Features:
                           feature  importance
0     air_temperature_at_2m(deg_C)    0.476645
7                         hour_cos    0.169861
6                         hour_sin    0.103303
5                       is_weekend    0.078046
8                  day_of_week_sin    0.076930
4          wind_speed_at_10m(km/h)    0.019016
1       relative_humidity_at_2m(%)    0.015349
2    direct_solar_radiation(W/m^2)    0.013487
11                       month_cos    0.012458
3   diffuse_solar_radiation(W/m^2)    0.012153
Feature importance results saved to explore/06_analysis/random_forest_merged_L14.B04_1H_cleaned.csv

Analyzing file: merged_L14.B03_1H_cleaned.csv

Top 10 Important Features:
                           feature  importance
0     air_temperature_at_2m(deg_C)    0.228180
7                         hour_cos    0.181586
6                         hour_sin    0.176720
8                  day_of_week_sin    0.09396

In [152]:
# PCA analysis

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def analyze_pca(input_folder="explore/05_cleaned/"):
    for filename in os.listdir(input_folder):
        if filename.endswith("_cleaned.csv"):
            file_path = os.path.join(input_folder, filename)
            print(f"\nAnalyzing file: {filename}")
            
            # Load the CSV file
            df = pd.read_csv(file_path)
            
            # Separate features and target
            X = df.drop(['timestamp', 'total_load(kW)'], axis=1)
            y = df['total_load(kW)']
            
            # Standardize the features
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
            
            # Principal Component Analysis
            pca = PCA()
            pca_result = pca.fit_transform(X_scaled)

            # Get the loadings
            loadings = pd.DataFrame(
                pca.components_.T,
                columns=[f'PC{i+1}' for i in range(pca.n_components_)],
                index=X.columns
            )
            
            # Print the loadings for the first 8 components
            print(loadings.iloc[:, :8])
            
            # Optionally, print the explained variance ratio
            print("\nExplained variance ratio:")
            print(pca.explained_variance_ratio_[:8])
            
            # Calculate cumulative explained variance ratio
            cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
            
            # Visualize PCA results
            plt.figure(figsize=(10, 6))
            plt.plot(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio, 'bo-')
            plt.xlabel('Number of Components')
            plt.ylabel('Cumulative Explained Variance Ratio')
            plt.title(f'PCA: Cumulative Explained Variance Ratio - {filename}')
            plt.grid(True)
            plt.tight_layout()
            plt.savefig(f'explore/06_analysis/pca_explained_variance_{filename[:-4]}.png')
            plt.close()
            
            # Determine number of components for 95% variance explained
            n_components_95 = np.argmax(cumulative_variance_ratio >= 0.95) + 1
            
            print(f"Number of components explaining 95% of variance: {n_components_95}")
            
            # Visualize first two principal components
            plt.figure(figsize=(10, 8))
            scatter = plt.scatter(pca_result[:, 0], pca_result[:, 1], c=y, cmap='viridis', alpha=0.5)
            plt.colorbar(scatter, label='Total Load (kW)')
            plt.xlabel('First Principal Component')
            plt.ylabel('Second Principal Component')
            plt.title(f'PCA: First Two Principal Components - {filename}')
            plt.tight_layout()
            plt.savefig(f'explore/06_analysis/pca_scatter_{filename[:-4]}.png')
            plt.close()

# Run the analysis
analyze_pca()


Analyzing file: merged_L14.B04_1H_cleaned.csv
                                     PC1       PC2       PC3       PC4  \
air_temperature_at_2m(deg_C)    0.377284 -0.040381 -0.468324 -0.053354   
relative_humidity_at_2m(%)     -0.417156 -0.025259  0.041307  0.086831   
direct_solar_radiation(W/m^2)   0.072728 -0.098538 -0.189248  0.635297   
diffuse_solar_radiation(W/m^2) -0.116128  0.121193 -0.239797 -0.179748   
wind_speed_at_10m(km/h)         0.432451  0.029818  0.117524 -0.068046   
is_weekend                      0.003551  0.696777 -0.028351  0.068879   
hour_sin                       -0.017752  0.014798  0.152542 -0.231990   
hour_cos                       -0.312137 -0.005186 -0.407590 -0.332179   
day_of_week_sin                -0.001731 -0.676730  0.018742 -0.085053   
day_of_week_cos                -0.005732  0.165080 -0.038205 -0.038464   
month_sin                      -0.073809  0.031808  0.545828 -0.381217   
month_cos                      -0.321853 -0.000941  0.310009  0.4

In [157]:
# Combine the 14 random forest csv results into one csv file

import glob

# Set the input and output directories
input_dir = "explore/06_analysis/"
output_dir = "explore/06_analysis/"

# Set the output filename
output_filename = "random-forest-results.csv"

# Get a list of all CSV files in the input directory
csv_files = glob.glob(os.path.join(input_dir, "random_forest_*.csv"))

# Check if we have exactly 14 CSV files
if len(csv_files) != 14:
    print(f"Warning: Found {len(csv_files)} CSV files instead of 14.")

# Initialize an empty list to store dataframes
dfs = []

# Read each CSV file and append it to the list
for file in csv_files:
    df = pd.read_csv(file)
    
    # Check if the required columns are present
    required_columns = ['filename', 'feature_number', 'feature_name', 'importance']
    if not all(col in df.columns for col in required_columns):
        print(f"Warning: File {file} is missing one or more required columns.")
        continue
    
    dfs.append(df)

# Concatenate all dataframes vertically
result = pd.concat(dfs, axis=0, ignore_index=True)
print('Result shape:', result.shape)
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Write the result to the output CSV file
output_path = os.path.join(output_dir, output_filename)
result.to_csv(output_path, index=False)

print(f"Concatenation complete. Output file saved as {output_path}")

Result shape: (196, 4)
Concatenation complete. Output file saved as explore/06_analysis/random-forest-results.csv


In [158]:
# Load disaggregation: KMeans

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Load random forest results
rf_results = pd.read_csv('explore/06_analysis/random-forest-results.csv', sep=',')

def get_top_features(filename, n_features=5):
    file_results = rf_results[rf_results['filename'] == filename]
    top_features = file_results.nlargest(n_features, 'importance')
    return dict(zip(top_features['feature_name'], top_features['importance']))

def k_means(file_path):
    filename = os.path.basename(file_path)
    
    # Get top features and their importances
    top_features = get_top_features(filename)
    
    # Load the data
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)
    
    # Select features based on random forest results
    X = df[list(top_features.keys()) + ['total_load(kW)']]
    
    # Normalize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Apply weights to the scaled features
    weights = np.array(list(top_features.values()) + [1.0])  # weight for total_load is 1
    X_weighted = X_scaled * weights
    
    # Apply K-means clustering
    kmeans = KMeans(n_clusters=3, random_state=42)
    df['cluster'] = kmeans.fit_predict(X_weighted)
    
    # Disaggregation based on filename and temperature
    if "L09.B01" in filename or "L10.B01" in filename:
        temp_threshold_cool = 25  # °C
        temp_threshold_heat = 15  # °C
        
        df['load_type'] = np.select(
            [df['air_temperature_at_2m(deg_C)'] > temp_threshold_cool,
             df['air_temperature_at_2m(deg_C)'] < temp_threshold_heat],
            ['cooling', 'heating'],
            default='baseline'
        )
        
        for load_type in ['cooling', 'heating']:
            mask = (df['load_type'] == load_type) & (df['cluster'] == df.groupby('load_type')['total_load(kW)'].transform('mean').idxmax())
            df[f'{load_type}_load'] = np.where(mask, df['total_load(kW)'], 0)
        
        df['Temperature_dependent(kW)'] = df['heating_load'] + df['cooling_load']
    else:
        temp_threshold_heat = 15  # °C
        
        df['load_type'] = np.where(df['air_temperature_at_2m(deg_C)'] < temp_threshold_heat, 'heating', 'baseline')
        
        heating_cluster = df[df['load_type'] == 'heating']['cluster'].mode().iloc[0]
        mask = (df['load_type'] == 'heating') & (df['cluster'] == heating_cluster)
        df['Temperature_dependent(kW)'] = np.where(mask, df['total_load(kW)'], 0)
    
    # Prepare output dataframe
    output_df = df.reset_index()[['timestamp', 'Temperature_dependent(kW)']]
    
    # Create output directory if it doesn't exist
    output_dir = 'explore/07_results/kmeans'
    os.makedirs(output_dir, exist_ok=True)
    
    # Save to CSV
    output_path = os.path.join(output_dir, filename)
    output_df.to_csv(output_path, index=False)
    
    print(f"Processed and saved: {output_path}")

# Process all CSV files in the input directory
input_dir = 'explore/05_cleaned/'  # Replace with your actual input directory path
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        file_path = os.path.join(input_dir, filename)
        k_means(file_path)

Processed and saved: explore/07_results/kmeans/merged_L14.B04_1H_cleaned.csv
Processed and saved: explore/07_results/kmeans/merged_L14.B03_1H_cleaned.csv
Processed and saved: explore/07_results/kmeans/merged_L14.B02_1H_cleaned.csv
Processed and saved: explore/07_results/kmeans/merged_L10.B01_30min_cleaned.csv
Processed and saved: explore/07_results/kmeans/merged_L14.B05_1H_cleaned.csv
Processed and saved: explore/07_results/kmeans/merged_L14.B01_1H_cleaned.csv
Processed and saved: explore/07_results/kmeans/merged_L09.B01_15min_cleaned.csv
Processed and saved: explore/07_results/kmeans/merged_L09.B01_1H_cleaned.csv
Processed and saved: explore/07_results/kmeans/merged_L10.B01_15min_cleaned.csv
Processed and saved: explore/07_results/kmeans/merged_L06.B01_1H_cleaned.csv
Processed and saved: explore/07_results/kmeans/merged_L03.B02_1H_cleaned.csv
Processed and saved: explore/07_results/kmeans/merged_L10.B01_1H_cleaned.csv
Processed and saved: explore/07_results/kmeans/merged_L09.B01_30min

In [159]:
# Rename files to submit

import re

def rename_csv_files(directory):
    # Compile the regex pattern
    pattern = re.compile(r'^merged_(.+)_cleaned\.csv$')

    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        # Check if the file matches our pattern
        match = pattern.match(filename)
        if match:
            # Extract the middle part of the filename
            new_name = match.group(1) + '.csv'
            
            # Construct full file paths
            old_path = os.path.join(directory, filename)
            new_path = os.path.join(directory, new_name)
            
            # Rename the file
            os.rename(old_path, new_path)
            print(f'Renamed: {filename} -> {new_name}')

# Specify the directory containing the CSV files
directory = 'explore/07_results/kmeans/'  # Replace this with your actual directory path

# Call the function
rename_csv_files(directory)

print("Renaming process completed.")

Renamed: merged_L14.B04_1H_cleaned.csv -> L14.B04_1H.csv
Renamed: merged_L14.B03_1H_cleaned.csv -> L14.B03_1H.csv
Renamed: merged_L14.B02_1H_cleaned.csv -> L14.B02_1H.csv
Renamed: merged_L10.B01_30min_cleaned.csv -> L10.B01_30min.csv
Renamed: merged_L14.B05_1H_cleaned.csv -> L14.B05_1H.csv
Renamed: merged_L14.B01_1H_cleaned.csv -> L14.B01_1H.csv
Renamed: merged_L09.B01_15min_cleaned.csv -> L09.B01_15min.csv
Renamed: merged_L09.B01_1H_cleaned.csv -> L09.B01_1H.csv
Renamed: merged_L10.B01_15min_cleaned.csv -> L10.B01_15min.csv
Renamed: merged_L06.B01_1H_cleaned.csv -> L06.B01_1H.csv
Renamed: merged_L03.B02_1H_cleaned.csv -> L03.B02_1H.csv
Renamed: merged_L10.B01_1H_cleaned.csv -> L10.B01_1H.csv
Renamed: merged_L09.B01_30min_cleaned.csv -> L09.B01_30min.csv
Renamed: merged_L09.B01_5min_cleaned.csv -> L09.B01_5min.csv
Renaming process completed.


In [161]:
# Load disaggregation: NMF

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import NMF

# Load random forest results
def load_rf_results(rf_results_path):
    rf_results = pd.read_csv(rf_results_path)
    return rf_results


# Prepare data for a specific file
def prepare_data(file_path, rf_results, num_features):
    # Extract building name from file path
    building_name = os.path.basename(file_path).replace('merged_', '').replace('_cleaned.csv', '')
    
    # Get feature importances for this building
    building_importances = rf_results[rf_results['filename'] == f"merged_{building_name}_cleaned.csv"]
    top_features = building_importances.nlargest(num_features, 'importance')['feature_name'].tolist()
    
    # Load data
    df = pd.read_csv(file_path)
    
    # Select relevant features
    X = df[top_features]
    y = df['total_load(kW)']
    timestamp = df['timestamp']
    
    return X, y, timestamp, building_name, top_features


def normalize_and_scale_features(X):
    if X.empty:
        return None, None, None
    
    # First, normalize the data (mean=0, variance=1)
    normalizer = StandardScaler()
    X_normalized = normalizer.fit_transform(X)
    
    # Then, scale to [0, 1] range
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X_normalized)
    
    return X_scaled, normalizer, scaler



# Scale features
def scale_features(X):
    if X.empty:
        return None, None
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, scaler

# Apply NMF
def apply_nmf(X, n_components):
    if X is None:
        return None, None, None
    model = NMF(n_components=n_components, init='random', random_state=0)
    W = model.fit_transform(X)
    H = model.components_
    return W, H, model

# Process a single file
def process_file(file_path, rf_results, num_features, output_dir):
    X, y, timestamp, building_name, top_features = prepare_data(file_path, rf_results, num_features)
    
    if X.empty:
        print(f"No features selected for {building_name}. Skipping this file.")
        return
    
    X_scaled, normalizer, scaler = normalize_and_scale_features(X)
    
    if X_scaled is None:
        print(f"Error scaling features for {building_name}. Skipping this file.")
        return
    
    # Determine number of components based on building name
    n_components = 2 if building_name.startswith(('L09.B01', 'L10.B01')) else 1
    
    W, H, model = apply_nmf(X_scaled, n_components)
    
    if W is None:
        print(f"Error applying NMF for {building_name}. Skipping this file.")
        return
    
    total_load_reconstructed = analyze_results(W, H, top_features, scaler, building_name)
    
    # Calculate temperature-dependent load
    temperature_dependent_load = np.sum(W, axis=1) if n_components == 2 else W.squeeze()
    
    # Prepare output
    output_df = pd.DataFrame({
        'timestamp': timestamp,
        'Temperature_dependent(kW)': temperature_dependent_load
    })
    
    # Save output
    output_filename = f"{building_name}.csv"
    output_path = os.path.join(output_dir, output_filename)
    output_df.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

    # Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(y, label='Original Total Load')
    plt.plot(total_load_reconstructed, label='Reconstructed Total Load')
    plt.plot(temperature_dependent_load, label='Temperature Dependent Load')
    plt.legend()
    plt.title(f'Load Disaggregation for {building_name}')
    plt.xlabel('Time')
    plt.ylabel('Load (kW)')
    plt.savefig(os.path.join(output_dir, f"{building_name}_plot.png"))
    plt.close()


# Main execution
def main(input_dir, rf_results_path, num_features, output_dir):
    rf_results = load_rf_results(rf_results_path)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Process each file in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith('_cleaned.csv'):
            file_path = os.path.join(input_dir, filename)
            process_file(file_path, rf_results, num_features, output_dir)

# Run the script
input_dir = 'explore/05_cleaned/'
rf_results_path = 'explore/06_analysis/random-forest-results.csv'
num_features = 14  # Adjust this to include more or fewer features
output_dir = 'explore/07_results/NMF7'

main(input_dir, rf_results_path, num_features, output_dir)

Component 1 for L14.B04_1H:
  wind_direction_at_10m(deg)_cos: 1.769
  relative_humidity_at_2m(%): 1.565
  wind_direction_at_10m(deg)_sin: 1.351
  hour_cos: 1.137
  month_cos: 1.122
  diffuse_solar_radiation(W/m^2): 1.119
  hour_sin: 1.104
  day_of_week_sin: 1.082
  month_sin: 1.082
  day_of_week_cos: 1.059
  air_temperature_at_2m(deg_C): 0.984
  direct_solar_radiation(W/m^2): 0.618
  is_weekend: 0.612
  wind_speed_at_10m(km/h): 0.186

Results saved to explore/07_results/NMF7/L14.B04_1H.csv
Component 1 for L14.B03_1H:
  wind_direction_at_10m(deg)_cos: 1.625
  relative_humidity_at_2m(%): 1.438
  wind_direction_at_10m(deg)_sin: 1.242
  hour_cos: 1.044
  month_cos: 1.031
  diffuse_solar_radiation(W/m^2): 1.028
  hour_sin: 1.014
  day_of_week_sin: 0.994
  month_sin: 0.994
  day_of_week_cos: 0.973
  air_temperature_at_2m(deg_C): 0.904
  direct_solar_radiation(W/m^2): 0.568
  is_weekend: 0.563
  wind_speed_at_10m(km/h): 0.171

Results saved to explore/07_results/NMF7/L14.B03_1H.csv
Component 

In [112]:
input_dir = 'explore/05_cleaned'
for filename in os.listdir(input_dir):
    filepath = os.path.join(input_dir, filename)
    file = pd.read_csv(filepath)
    print(file.shape)

(8760, 16)
(8760, 16)
(8760, 16)
(27728, 16)
(8760, 16)
(8472, 16)
(32879, 16)
(8287, 16)
(55455, 16)
(3916, 16)
(8282, 16)
(13866, 16)
(16547, 16)
(95735, 16)


In [120]:
# Load disaggregation: NMF

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import NMF


def prepare_data(file_path, rf_results, num_features):
    building_name = os.path.basename(file_path).replace('merged_', '').replace('_cleaned.csv', '')
    building_importances = rf_results[rf_results['filename'] == f"merged_{building_name}_cleaned.csv"]
    top_features = building_importances.nlargest(num_features, 'importance')['feature_name'].tolist()
    
    df = pd.read_csv(file_path)
    X = df[top_features]
    y = df['total_load(kW)']
    timestamp = df['timestamp']
    
    return X, y, timestamp, building_name, top_features

def scale_features(X):
    if X.empty:
        return None, None
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, scaler

def apply_nmf(X, n_components):
    if X is None:
        return None, None, None
    model = NMF(n_components=n_components, init='random', random_state=0, max_iter=1000)
    W = model.fit_transform(X)
    H = model.components_
    return W, H, model

def process_file(file_path, rf_results, num_features, output_dir):
    X, y, timestamp, building_name, top_features = prepare_data(file_path, rf_results, num_features)
    
    if X.empty:
        print(f"No features selected for {building_name}. Skipping this file.")
        return
    
    X_scaled, scaler = scale_features(X)
    
    if X_scaled is None:
        print(f"Error scaling features for {building_name}. Skipping this file.")
        return
    
    # Use more components for all buildings
    n_components = 3
    
    W, H, model = apply_nmf(X_scaled, n_components)
    
    if W is None:
        print(f"Error applying NMF for {building_name}. Skipping this file.")
        return
    
    # Reconstruct the load
    reconstructed_load = np.dot(W, H)
    reconstructed_load = scaler.inverse_transform(reconstructed_load)
    
    # Sum all components for temperature-dependent load
    temperature_dependent_load = np.sum(reconstructed_load, axis=1)
    
    # Prepare output
    output_df = pd.DataFrame({
        'timestamp': timestamp,
        'Temperature_dependent(kW)': temperature_dependent_load
    })
    
    # Save output
    output_filename = f"{building_name}.csv"
    output_path = os.path.join(output_dir, output_filename)
    output_df.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

    # Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(y, label='Original Total Load', alpha=0.7)
    plt.plot(temperature_dependent_load, label='Temperature Dependent Load', alpha=0.7)
    for i in range(n_components):
        plt.plot(reconstructed_load[:, i], label=f'Component {i+1}', alpha=0.5)
    plt.legend()
    plt.title(f'Load Disaggregation for {building_name}')
    plt.xlabel('Time')
    plt.ylabel('Load (kW)')
    plt.savefig(os.path.join(output_dir, f"{building_name}_plot.png"))
    plt.close()

# Main function remains the same
def main(input_dir, rf_results_path, num_features, output_dir):
    rf_results = pd.read_csv(rf_results_path)
    os.makedirs(output_dir, exist_ok=True)
    
    for filename in os.listdir(input_dir):
        if filename.endswith('_cleaned.csv'):
            file_path = os.path.join(input_dir, filename)
            process_file(file_path, rf_results, num_features, output_dir)

# Run the script
input_dir = 'explore/05_cleaned/'
rf_results_path = 'explore/06_analysis/random-forest-results.csv'
num_features = 14  # Adjust this to include more or fewer features
output_dir = 'explore/07_results/NMF7'

main(input_dir, rf_results_path, num_features, output_dir)

Results saved to explore/07_results/NMF7/L14.B04_1H.csv
Results saved to explore/07_results/NMF7/L14.B03_1H.csv
Results saved to explore/07_results/NMF7/L14.B02_1H.csv
Results saved to explore/07_results/NMF7/L10.B01_30min.csv
Results saved to explore/07_results/NMF7/L14.B05_1H.csv
Results saved to explore/07_results/NMF7/L14.B01_1H.csv
Results saved to explore/07_results/NMF7/L09.B01_15min.csv
Results saved to explore/07_results/NMF7/L09.B01_1H.csv
Results saved to explore/07_results/NMF7/L10.B01_15min.csv
Results saved to explore/07_results/NMF7/L06.B01_1H.csv
Results saved to explore/07_results/NMF7/L03.B02_1H.csv
Results saved to explore/07_results/NMF7/L10.B01_1H.csv
Results saved to explore/07_results/NMF7/L09.B01_30min.csv
Results saved to explore/07_results/NMF7/L09.B01_5min.csv


In [160]:
# Load disaggregation: NMF

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import NMF

def prepare_data(file_path, rf_results, num_features):
    building_name = os.path.basename(file_path).replace('merged_', '').replace('_cleaned.csv', '')
    building_importances = rf_results[rf_results['filename'] == f"merged_{building_name}_cleaned.csv"]
    top_features = building_importances.nlargest(num_features, 'importance')['feature_name'].tolist()
    
    df = pd.read_csv(file_path)
    X = df[top_features]
    y = df['total_load(kW)']
    timestamp = df['timestamp']
    
    return X, y, timestamp, building_name, top_features

def scale_features(X):
    if X.empty:
        return None, None
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, scaler

def apply_nmf(X, n_components):
    if X is None:
        return None, None, None
    model = NMF(n_components=n_components, init='random', random_state=0, max_iter=1000)
    W = model.fit_transform(X)
    H = model.components_
    return W, H, model

def process_file(file_path, rf_results, num_features, output_dir):
    X, y, timestamp, building_name, top_features = prepare_data(file_path, rf_results, num_features)
    
    if X.empty:
        print(f"No features selected for {building_name}. Skipping this file.")
        return
    
    X_scaled, scaler = scale_features(X)
    
    if X_scaled is None:
        print(f"Error scaling features for {building_name}. Skipping this file.")
        return
    
    # Determine number of components based on building name
    n_components = 2 if building_name.startswith(('L09.B01', 'L10.B01')) else 1
    
    W, H, model = apply_nmf(X_scaled, n_components)
    
    if W is None:
        print(f"Error applying NMF for {building_name}. Skipping this file.")
        return
    
    # Reconstruct the load
    reconstructed_load = np.dot(W, H)
    
    # Identify the temperature-dependent component
    temp_feature_index = top_features.index('air_temperature_at_2m(deg_C)')
    temp_component_index = np.argmax(H[:, temp_feature_index])
    
    # Extract the temperature-dependent load
    temperature_dependent_load = W[:, temp_component_index] * np.max(H[temp_component_index])
    
    # Scale the temperature-dependent load to match the original load range
    load_min, load_max = np.min(y), np.max(y)
    temperature_dependent_load = (temperature_dependent_load - np.min(temperature_dependent_load)) / (np.max(temperature_dependent_load) - np.min(temperature_dependent_load))
    temperature_dependent_load = temperature_dependent_load * (load_max - load_min) + load_min
    
    # Ensure temperature-dependent load is between 0 and total load
    temperature_dependent_load = np.clip(temperature_dependent_load, 0, y.values)
    
    # Prepare output
    output_df = pd.DataFrame({
        'timestamp': timestamp,
        'Temperature_dependent(kW)': temperature_dependent_load
    })
    
    # Save output
    output_filename = f"{building_name}.csv"
    output_path = os.path.join(output_dir, output_filename)
    output_df.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

    # Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(y, label='Original Total Load', alpha=0.7)
    plt.plot(temperature_dependent_load, label='Temperature Dependent Load', alpha=0.7)
    for i in range(n_components):
        component_load = W[:, i] * np.max(H[i])
        plt.plot(component_load, label=f'Component {i+1}', alpha=0.5)
    plt.legend()
    plt.title(f'Load Disaggregation for {building_name}')
    plt.xlabel('Time')
    plt.ylabel('Load (kW)')
    plt.savefig(os.path.join(output_dir, f"{building_name}_plot.png"))
    plt.close()


# Main function remains the same
def main(input_dir, rf_results_path, num_features, output_dir):
    rf_results = pd.read_csv(rf_results_path)
    os.makedirs(output_dir, exist_ok=True)
    
    for filename in os.listdir(input_dir):
        if filename.endswith('_cleaned.csv'):
            file_path = os.path.join(input_dir, filename)
            process_file(file_path, rf_results, num_features, output_dir)

# Run the script
input_dir = 'explore/05_cleaned/'
rf_results_path = 'explore/06_analysis/random-forest-results.csv'
num_features = 14  # Adjust this to include more or fewer features
output_dir = 'explore/07_results/NMF7'

main(input_dir, rf_results_path, num_features, output_dir)

Results saved to explore/07_results/NMF7/L14.B04_1H.csv
Results saved to explore/07_results/NMF7/L14.B03_1H.csv
Results saved to explore/07_results/NMF7/L14.B02_1H.csv
Results saved to explore/07_results/NMF7/L10.B01_30min.csv
Results saved to explore/07_results/NMF7/L14.B05_1H.csv
Results saved to explore/07_results/NMF7/L14.B01_1H.csv
Results saved to explore/07_results/NMF7/L09.B01_15min.csv
Results saved to explore/07_results/NMF7/L09.B01_1H.csv
Results saved to explore/07_results/NMF7/L10.B01_15min.csv
Results saved to explore/07_results/NMF7/L06.B01_1H.csv
Results saved to explore/07_results/NMF7/L03.B02_1H.csv
Results saved to explore/07_results/NMF7/L10.B01_1H.csv
Results saved to explore/07_results/NMF7/L09.B01_30min.csv
Results saved to explore/07_results/NMF7/L09.B01_5min.csv


In [166]:
# Load disaggregation: NMF seasonal influence
from datetime import datetime 

def prepare_data(file_path, rf_results, num_features):
    building_name = os.path.basename(file_path).replace('merged_', '').replace('_cleaned.csv', '')
    building_importances = rf_results[rf_results['filename'] == f"merged_{building_name}_cleaned.csv"]
    top_features = building_importances.nlargest(num_features, 'importance')['feature_name'].tolist()
    
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['season'] = df['timestamp'].dt.month.map({1: 'Winter', 2: 'Winter', 3: 'Spring', 
                                                 4: 'Spring', 5: 'Spring', 6: 'Summer', 
                                                 7: 'Summer', 8: 'Summer', 9: 'Autumn', 
                                                 10: 'Autumn', 11: 'Autumn', 12: 'Winter'})
    
    X = df[top_features]
    y = df['total_load(kW)']
    timestamp = df['timestamp']
    season = df['season']
    
    return X, y, timestamp, season, building_name, top_features

def scale_features(X):
    if X.empty:
        return None, None
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, scaler

def apply_nmf(X, n_components):
    if X is None:
        return None, None, None
    model = NMF(n_components=n_components, init='random', random_state=0, max_iter=1000)
    W = model.fit_transform(X)
    H = model.components_
    return W, H, model

def process_file(file_path, rf_results, num_features, output_dir):
    X, y, timestamp, season, building_name, top_features = prepare_data(file_path, rf_results, num_features)
    
    if X.empty:
        print(f"No features selected for {building_name}. Skipping this file.")
        return
    
    X_scaled, scaler = scale_features(X)
    
    if X_scaled is None:
        print(f"Error scaling features for {building_name}. Skipping this file.")
        return
    
    # Determine number of components based on building name
    n_components = 2 if building_name.startswith(('L09.B01', 'L10.B01')) else 1
    
    # Process each season separately
    seasons = ['Winter', 'Spring', 'Summer', 'Autumn']
    temperature_dependent_load = np.zeros(len(y))
    
    for s in seasons:
        season_mask = season == s
        X_season = X_scaled[season_mask]
        y_season = y[season_mask]
        
        W, H, model = apply_nmf(X_season, n_components)
        
        if W is None:
            print(f"Error applying NMF for {building_name} in {s}. Skipping this season.")
            continue
        
        # Identify the temperature-dependent component
        temp_feature_index = top_features.index('air_temperature_at_2m(deg_C)')
        temp_component_index = np.argmax(H[:, temp_feature_index])
        
        # Extract the temperature-dependent load for this season
        season_temp_load = W[:, temp_component_index] * np.max(H[temp_component_index])
        
        # Scale the temperature-dependent load to match the original load range for this season
        load_min, load_max = np.min(y_season), np.max(y_season)
        season_temp_load = (season_temp_load - np.min(season_temp_load)) / (np.max(season_temp_load) - np.min(season_temp_load))
        season_temp_load = season_temp_load * (load_max - load_min) + load_min
        
        # Assign the seasonal temperature-dependent load to the full array
        temperature_dependent_load[season_mask] = season_temp_load
    
    # Ensure temperature-dependent load is between 0 and total load
    temperature_dependent_load = np.clip(temperature_dependent_load, 0, y.values)
    
    # Prepare output
    output_df = pd.DataFrame({
        'timestamp': timestamp,
        'Temperature_dependent(kW)': temperature_dependent_load
    })
    
    # Save output
    output_filename = f"{building_name}.csv"
    output_path = os.path.join(output_dir, output_filename)
    output_df.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

    # Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(timestamp, y, label='Original Total Load', alpha=0.7)
    plt.plot(timestamp, temperature_dependent_load, label='Temperature Dependent Load', alpha=0.7)
    plt.legend()
    plt.title(f'Load Disaggregation for {building_name}')
    plt.xlabel('Time')
    plt.ylabel('Load (kW)')
    plt.savefig(os.path.join(output_dir, f"{building_name}_plot.png"))
    plt.close()

# The main function remains the same
def main(input_dir, rf_results_path, num_features, output_dir):
    rf_results = pd.read_csv(rf_results_path)
    os.makedirs(output_dir, exist_ok=True)
    
    for filename in os.listdir(input_dir):
        if filename.endswith('_cleaned.csv'):
            file_path = os.path.join(input_dir, filename)
            process_file(file_path, rf_results, num_features, output_dir)

# Run the script
input_dir = 'explore/05_cleaned/'
rf_results_path = 'explore/06_analysis/random-forest-results.csv'
num_features = 10  # Using all features
output_dir = 'explore/07_results/NMF7'

main(input_dir, rf_results_path, num_features, output_dir)

Results saved to explore/07_results/NMF7/L14.B04_1H.csv
Results saved to explore/07_results/NMF7/L14.B03_1H.csv
Results saved to explore/07_results/NMF7/L14.B02_1H.csv
Results saved to explore/07_results/NMF7/L10.B01_30min.csv
Results saved to explore/07_results/NMF7/L14.B05_1H.csv
Results saved to explore/07_results/NMF7/L14.B01_1H.csv
Results saved to explore/07_results/NMF7/L09.B01_15min.csv
Results saved to explore/07_results/NMF7/L09.B01_1H.csv
Results saved to explore/07_results/NMF7/L10.B01_15min.csv
Results saved to explore/07_results/NMF7/L06.B01_1H.csv
Results saved to explore/07_results/NMF7/L03.B02_1H.csv
Results saved to explore/07_results/NMF7/L10.B01_1H.csv
Results saved to explore/07_results/NMF7/L09.B01_30min.csv
Results saved to explore/07_results/NMF7/L09.B01_5min.csv


In [170]:
# Load disaggregation: NMF monthly influence

from datetime import datetime

def prepare_data(file_path, rf_results, num_features):
    building_name = os.path.basename(file_path).replace('merged_', '').replace('_cleaned.csv', '')
    building_importances = rf_results[rf_results['filename'] == f"merged_{building_name}_cleaned.csv"]
    top_features = building_importances.nlargest(num_features, 'importance')['feature_name'].tolist()
    
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['month'] = df['timestamp'].dt.month
    
    X = df[top_features]
    y = df['total_load(kW)']
    timestamp = df['timestamp']
    month = df['month']
    
    return X, y, timestamp, month, building_name, top_features

def scale_features(X):
    if X.empty:
        return None, None
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, scaler

def apply_nmf(X, n_components):
    if X is None or X.shape[0] == 0:
        return None, None, None
    model = NMF(n_components=n_components, init='random', random_state=0, max_iter=1000)
    W = model.fit_transform(X)
    H = model.components_
    return W, H, model

def process_file(file_path, rf_results, num_features, output_dir):
    X, y, timestamp, month, building_name, top_features = prepare_data(file_path, rf_results, num_features)
    
    if X.empty:
        print(f"No features selected for {building_name}. Skipping this file.")
        return
    
    X_scaled, scaler = scale_features(X)
    
    if X_scaled is None:
        print(f"Error scaling features for {building_name}. Skipping this file.")
        return
    
    # Determine number of components based on building name
    n_components = 2 if building_name.startswith(('L09.B01', 'L10.B01')) else 1
    
    # Process each month separately
    temperature_dependent_load = np.zeros(len(y))
    
    for m in range(1, 13):
        month_mask = month == m
        X_month = X_scaled[month_mask]
        y_month = y[month_mask]
        
        if X_month.shape[0] == 0:
            print(f"No data for month {m} in {building_name}. Skipping this month.")
            continue
        
        W, H, model = apply_nmf(X_month, n_components)
        
        if W is None:
            print(f"Error applying NMF for {building_name} in month {m}. Skipping this month.")
            continue
        
        # Identify the temperature-dependent component
        temp_feature_index = top_features.index('air_temperature_at_2m(deg_C)')
        temp_component_index = np.argmax(H[:, temp_feature_index])
        
        # Extract the temperature-dependent load for this month
        month_temp_load = W[:, temp_component_index] * np.max(H[temp_component_index])
        
        # Scale the temperature-dependent load to match the original load range for this month
        load_min, load_max = np.min(y_month), np.max(y_month)
        month_temp_load = (month_temp_load - np.min(month_temp_load)) / (np.max(month_temp_load) - np.min(month_temp_load))
        month_temp_load = month_temp_load * (load_max - load_min) + load_min
        
        # Assign the monthly temperature-dependent load to the full array
        temperature_dependent_load[month_mask] = month_temp_load
    
    # Ensure temperature-dependent load is between 0 and total load
    temperature_dependent_load = np.clip(temperature_dependent_load, 0, y.values)
    
    # Prepare output
    output_df = pd.DataFrame({
        'timestamp': timestamp,
        'Temperature_dependent(kW)': temperature_dependent_load
    })
    
    # Save output
    output_filename = f"{building_name}.csv"
    output_path = os.path.join(output_dir, output_filename)
    output_df.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

    # Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(timestamp, y, label='Original Total Load', alpha=0.7)
    plt.plot(timestamp, temperature_dependent_load, label='Temperature Dependent Load', alpha=0.7)
    plt.legend()
    plt.title(f'Load Disaggregation for {building_name}')
    plt.xlabel('Time')
    plt.ylabel('Load (kW)')
    plt.savefig(os.path.join(output_dir, f"{building_name}_plot.png"))
    plt.close()

# The main function remains the same
def main(input_dir, rf_results_path, num_features, output_dir):
    rf_results = pd.read_csv(rf_results_path)
    os.makedirs(output_dir, exist_ok=True)
    
    for filename in os.listdir(input_dir):
        if filename.endswith('_cleaned.csv'):
            file_path = os.path.join(input_dir, filename)
            process_file(file_path, rf_results, num_features, output_dir)

# Run the script
input_dir = 'explore/05_cleaned/'
rf_results_path = 'explore/06_analysis/random-forest-results.csv'
num_features = 8  # Using all features
output_dir = 'explore/07_results/NMF8'

main(input_dir, rf_results_path, num_features, output_dir)

Results saved to explore/07_results/NMF8/L14.B04_1H.csv
Results saved to explore/07_results/NMF8/L14.B03_1H.csv
Results saved to explore/07_results/NMF8/L14.B02_1H.csv
Results saved to explore/07_results/NMF8/L10.B01_30min.csv
Results saved to explore/07_results/NMF8/L14.B05_1H.csv
Results saved to explore/07_results/NMF8/L14.B01_1H.csv
Results saved to explore/07_results/NMF8/L09.B01_15min.csv
Results saved to explore/07_results/NMF8/L09.B01_1H.csv
Results saved to explore/07_results/NMF8/L10.B01_15min.csv
No data for month 3 in L06.B01_1H. Skipping this month.
Results saved to explore/07_results/NMF8/L06.B01_1H.csv
Results saved to explore/07_results/NMF8/L03.B02_1H.csv
Results saved to explore/07_results/NMF8/L10.B01_1H.csv
Results saved to explore/07_results/NMF8/L09.B01_30min.csv
Results saved to explore/07_results/NMF8/L09.B01_5min.csv


In [172]:
# Load disaggregation: k-means

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Set input and output directories
input_dir = 'explore/05_cleaned/'
output_dir = 'explore/07_results/kmeans2/'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load random forest results
rf_results = pd.read_csv('explore/06_analysis/random-forest-results.csv')

# Function to get top N features for a file
def get_top_features(filename, n=10):
    file_results = rf_results[rf_results['filename'] == filename]
    return file_results.nlargest(n, 'importance')['feature_name'].tolist()

# Function to normalize features
def normalize_features(df, features):
    scaler = StandardScaler()
    df[features] = scaler.fit_transform(df[features])
    return df

# Function to perform monthly K-means clustering
def monthly_kmeans(df, features, n_clusters=2):
    df['month'] = pd.to_datetime(df['timestamp']).dt.month
    results = []
    for month in range(1, 13):
        month_df = df[df['month'] == month]
        if len(month_df) > 0:
            kmeans = KMeans(n_clusters=n_clusters, random_state=42)
            clusters = kmeans.fit_predict(month_df[features])
            results.append(pd.DataFrame({
                'timestamp': month_df['timestamp'],
                'cluster': clusters,
                'total_load': month_df['total_load(kW)']
            }))
    return pd.concat(results).sort_values('timestamp')

# Function to estimate temperature-dependent load
def estimate_temp_dependent_load(df, n_clusters):
    temp_dependent = []
    for cluster in range(n_clusters):
        cluster_df = df[df['cluster'] == cluster]
        baseline = cluster_df['total_load'].min()
        temp_dependent.append(cluster_df['total_load'] - baseline)
    return pd.concat(temp_dependent).sort_index()

# Function to process a single file
def process_file(filename, n_features=10):
    print(f"Processing {filename}...")
    input_path = os.path.join(input_dir, filename)
    df = pd.read_csv(input_path)
    
    # Get top features for this file
    top_features = get_top_features(filename, n_features)
    
    # Normalize features
    df = normalize_features(df, top_features)
    
    # Perform monthly K-means clustering
    n_clusters = 3 if 'L09.B01' in filename or 'L10.B01' in filename else 2
    clustered = monthly_kmeans(df, top_features, n_clusters)
    
    # Estimate temperature-dependent load
    temp_dependent = estimate_temp_dependent_load(clustered, n_clusters)
    
    # Create output DataFrame
    output = pd.DataFrame({
        'timestamp': clustered['timestamp'],
        'Temperature_dependent(kW)': temp_dependent
    })
    
    # Check for invalid values
    invalid_mask = (output['Temperature_dependent(kW)'] < 0) | (output['Temperature_dependent(kW)'] > df['total_load(kW)'])
    if invalid_mask.any():
        print("Warning: Invalid temperature-dependent load values detected:")
        print(output[invalid_mask])
    
    # Clip values to valid range
    output['Temperature_dependent(kW)'] = output['Temperature_dependent(kW)'].clip(lower=0, upper=df['total_load(kW)'])
    
    # Save output
    output_filename = filename.replace('merged_', '').replace('_cleaned', '')
    output_path = os.path.join(output_dir, output_filename)
    output.to_csv(output_path, index=False)
    print(f"Saved output to {output_path}")



# Process all files
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        process_file(filename)

print("Processing complete.")

Processing merged_L14.B04_1H_cleaned.csv...
Saved output to explore/07results/kmeans2/L14.B04_1H.csv
Processing merged_L14.B03_1H_cleaned.csv...
Saved output to explore/07results/kmeans2/L14.B03_1H.csv
Processing merged_L14.B02_1H_cleaned.csv...
Saved output to explore/07results/kmeans2/L14.B02_1H.csv
Processing merged_L10.B01_30min_cleaned.csv...
Saved output to explore/07results/kmeans2/L10.B01_30min.csv
Processing merged_L14.B05_1H_cleaned.csv...
Saved output to explore/07results/kmeans2/L14.B05_1H.csv
Processing merged_L14.B01_1H_cleaned.csv...
Saved output to explore/07results/kmeans2/L14.B01_1H.csv
Processing merged_L09.B01_15min_cleaned.csv...
Saved output to explore/07results/kmeans2/L09.B01_15min.csv
Processing merged_L09.B01_1H_cleaned.csv...
Saved output to explore/07results/kmeans2/L09.B01_1H.csv
Processing merged_L10.B01_15min_cleaned.csv...
Saved output to explore/07results/kmeans2/L10.B01_15min.csv
Processing merged_L06.B01_1H_cleaned.csv...
Saved output to explore/07res

In [178]:
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import warnings

# Function to load and preprocess data
def load_and_preprocess(file_path):
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['month'] = df['timestamp'].dt.month
    return df

# Function to get feature importance from random forest results
def get_feature_importance(rf_results_path, filename):
    rf_results = pd.read_csv(rf_results_path)
    file_results = rf_results[rf_results['filename'] == filename]
    return dict(zip(file_results['feature_name'], file_results['importance']))

# Function to select top N features
def select_top_features(feature_importance, n=10):
    return sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)[:n]


def smooth_series(series, window=24):
    if isinstance(series, pd.Series):
        return series.rolling(window=window, center=True, min_periods=1).mean()
    elif isinstance(series, np.ndarray):
        return pd.Series(series).rolling(window=window, center=True, min_periods=1).mean().values
    else:
        raise TypeError("Input must be a pandas Series or numpy array")

def apply_relaxed_constraint(total_load, dependent_load, smoothing_window=24):
    smoothed_total = smooth_series(total_load, smoothing_window)
    smoothed_dependent = smooth_series(dependent_load, smoothing_window)
    
    # Calculate the ratio of smoothed dependent to total load
    ratio = np.clip(smoothed_dependent / smoothed_total, 0, 1)
    
    # Apply this ratio to the original dependent load
    constrained_load = dependent_load * ratio
    
    return constrained_load

def gmm_load_disaggregation(input_file, rf_results_path, output_dir, n_features=10):
    # Load and preprocess data
    df = load_and_preprocess(input_file)
    
    # Get feature importance
    filename = os.path.basename(input_file)
    feature_importance = get_feature_importance(rf_results_path, filename)
    
    # Select top N features
    top_features = select_top_features(feature_importance, n_features)
    selected_features = [f[0] for f in top_features]
    
    # Normalize features
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(df[selected_features])
    
    # Prepare for GMM training
    X = np.column_stack((normalized_features, df['total_load(kW)'].values))
    
    # Determine if we need to disaggregate both heating and cooling
    disaggregate_both = any(building in filename for building in ['L09.B01', 'L10.B01'])
    
    # Initialize heating and cooling loads
    heating_load = np.zeros(len(df))
    cooling_load = np.zeros(len(df))
    
    # Train GMM models for each month
    for month in range(1, 13):
        month_mask = df['month'] == month
        month_data = X[month_mask]
        
        if len(month_data) > 0:
            # For heating (temperature < 18°C)
            heating_mask = (df['air_temperature_at_2m(deg_C)'] < 18) & month_mask
            if np.any(heating_mask):
                heating_gmm = GaussianMixture(n_components=2, random_state=42)
                heating_gmm.fit(X[heating_mask])
                heating_predictions = heating_gmm.predict(X[heating_mask])
                heating_load[heating_mask] = heating_gmm.means_[heating_predictions, -1]
            
            # For cooling (temperature > 25°C), only if needed
            if disaggregate_both:
                cooling_mask = (df['air_temperature_at_2m(deg_C)'] > 25) & month_mask
                if np.any(cooling_mask):
                    cooling_gmm = GaussianMixture(n_components=2, random_state=42)
                    cooling_gmm.fit(X[cooling_mask])
                    cooling_predictions = cooling_gmm.predict(X[cooling_mask])
                    cooling_load[cooling_mask] = cooling_gmm.means_[cooling_predictions, -1]
    
    # Combine heating and cooling loads
    temp_dependent_load = heating_load + cooling_load if disaggregate_both else heating_load
    
    # Apply relaxed constraint
    temp_dependent_load = apply_relaxed_constraint(df['total_load(kW)'].values, temp_dependent_load)
    
    if disaggregate_both:
        heating_load = apply_relaxed_constraint(df['total_load(kW)'].values, heating_load)
        cooling_load = apply_relaxed_constraint(df['total_load(kW)'].values, cooling_load)
    
    # Prepare output
    output_df = pd.DataFrame({
        'timestamp': df['timestamp'],
        'Temperature_dependent(kW)': temp_dependent_load
    })
    
    if disaggregate_both:
        output_df['Heating_Load(kW)'] = heating_load
        output_df['Cooling_Load(kW)'] = cooling_load
    
    # Save output
    output_filename = filename.replace('merged_', '').replace('_cleaned', '')
    output_path = os.path.join(output_dir, output_filename)
    output_df.to_csv(output_path, index=False)
    
    print(f"Output saved to {output_path}")
    
    # Create and save visualizations
    create_visualizations(df, output_df, output_dir, output_filename, disaggregate_both)
    print(f"Visualizations saved to {output_dir}")

def create_visualizations(input_df, output_df, output_dir, filename, disaggregate_both):
    # Merge input and output dataframes
    merged_df = pd.merge(input_df, output_df, on='timestamp')
    
    # Time series plot
    plt.figure(figsize=(15, 6))
    plt.plot(merged_df['timestamp'], merged_df['total_load(kW)'], label='Total Load')
    plt.plot(merged_df['timestamp'], merged_df['Temperature_dependent(kW)'], label='Temperature-dependent Load')
    if disaggregate_both:
        plt.plot(merged_df['timestamp'], merged_df['Heating_Load(kW)'], label='Heating Load')
        plt.plot(merged_df['timestamp'], merged_df['Cooling_Load(kW)'], label='Cooling Load')
    plt.title(f'Load Disaggregation Results for {filename}')
    plt.xlabel('Timestamp')
    plt.ylabel('Load (kW)')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{filename}_timeseries.png'))
    plt.close()

    # Scatter plot
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(merged_df['air_temperature_at_2m(deg_C)'], 
                          merged_df['Temperature_dependent(kW)'],
                          c=merged_df['timestamp'].dt.month, 
                          cmap='viridis', 
                          alpha=0.5)
    plt.colorbar(scatter, label='Month')
    plt.axvline(x=18, color='r', linestyle='--', label='Heating Threshold (18°C)')
    if disaggregate_both:
        plt.axvline(x=25, color='g', linestyle='--', label='Cooling Threshold (25°C)')
    plt.title(f'Temperature-dependent Load vs Air Temperature for {filename}')
    plt.xlabel('Air Temperature (°C)')
    plt.ylabel('Temperature-dependent Load (kW)')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{filename}_scatter.png'))
    plt.close()

# Main execution
if __name__ == "__main__":
    input_dir = 'explore/05_cleaned/'
    rf_results_path = 'explore/06_analysis/random-forest-results.csv'
    output_dir = 'explore/07_results/gmm/'
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Process all files in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith('.csv'):
            input_file = os.path.join(input_dir, filename)
            print(f"Processing {filename}")
            gmm_load_disaggregation(input_file, rf_results_path, output_dir)

Processing merged_L14.B04_1H_cleaned.csv
Output saved to explore/07_results/gmm/L14.B04_1H.csv
Visualizations saved to explore/07_results/gmm/
Processing merged_L14.B03_1H_cleaned.csv
Output saved to explore/07_results/gmm/L14.B03_1H.csv
Visualizations saved to explore/07_results/gmm/
Processing merged_L14.B02_1H_cleaned.csv
Output saved to explore/07_results/gmm/L14.B02_1H.csv
Visualizations saved to explore/07_results/gmm/
Processing merged_L10.B01_30min_cleaned.csv
Output saved to explore/07_results/gmm/L10.B01_30min.csv
Visualizations saved to explore/07_results/gmm/
Processing merged_L14.B05_1H_cleaned.csv
Output saved to explore/07_results/gmm/L14.B05_1H.csv
Visualizations saved to explore/07_results/gmm/
Processing merged_L14.B01_1H_cleaned.csv
Output saved to explore/07_results/gmm/L14.B01_1H.csv


/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_78761/780327643.py:36: RuntimeWarning: divide by zero encountered in divide
  ratio = np.clip(smoothed_dependent / smoothed_total, 0, 1)
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_78761/780327643.py:36: RuntimeWarning: invalid value encountered in divide
  ratio = np.clip(smoothed_dependent / smoothed_total, 0, 1)


Visualizations saved to explore/07_results/gmm/
Processing merged_L09.B01_15min_cleaned.csv
Output saved to explore/07_results/gmm/L09.B01_15min.csv
Visualizations saved to explore/07_results/gmm/
Processing merged_L09.B01_1H_cleaned.csv
Output saved to explore/07_results/gmm/L09.B01_1H.csv
Visualizations saved to explore/07_results/gmm/
Processing merged_L10.B01_15min_cleaned.csv
Output saved to explore/07_results/gmm/L10.B01_15min.csv
Visualizations saved to explore/07_results/gmm/
Processing merged_L06.B01_1H_cleaned.csv
Output saved to explore/07_results/gmm/L06.B01_1H.csv


/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_78761/780327643.py:36: RuntimeWarning: divide by zero encountered in divide
  ratio = np.clip(smoothed_dependent / smoothed_total, 0, 1)


Visualizations saved to explore/07_results/gmm/
Processing merged_L03.B02_1H_cleaned.csv
Output saved to explore/07_results/gmm/L03.B02_1H.csv
Visualizations saved to explore/07_results/gmm/
Processing merged_L10.B01_1H_cleaned.csv
Output saved to explore/07_results/gmm/L10.B01_1H.csv
Visualizations saved to explore/07_results/gmm/
Processing merged_L09.B01_30min_cleaned.csv
Output saved to explore/07_results/gmm/L09.B01_30min.csv
Visualizations saved to explore/07_results/gmm/
Processing merged_L09.B01_5min_cleaned.csv
Output saved to explore/07_results/gmm/L09.B01_5min.csv
Visualizations saved to explore/07_results/gmm/


In [181]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
import seaborn as sns

def load_and_preprocess(file_path):
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['month'] = df['timestamp'].dt.month
    return df

def get_feature_importance(rf_results_path, filename):
    rf_results = pd.read_csv(rf_results_path)
    file_results = rf_results[rf_results['filename'] == filename]
    return dict(zip(file_results['feature_name'], file_results['importance']))

def select_top_features(feature_importance, n=10):
    return sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)[:n]

def xgboost_heating_load_disaggregation(input_file, rf_results_path, output_dir, n_features=10):
    # Load and preprocess data
    df = load_and_preprocess(input_file)
    
    # Create heating-specific features
    df['degrees_below_18'] = np.maximum(18 - df['air_temperature_at_2m(deg_C)'], 0)
    df['is_heating_season'] = df['month'].isin([1, 2, 3, 10, 11, 12]).astype(int)
    
    # Get feature importance
    filename = os.path.basename(input_file)
    feature_importance = get_feature_importance(rf_results_path, filename)
    
    # Select top N features, ensuring we include our new features
    top_features = select_top_features(feature_importance, n_features)
    selected_features = [f[0] for f in top_features] + ['degrees_below_18', 'is_heating_season']
    selected_features = list(set(selected_features))  # Remove duplicates
    
    # Prepare features
    X = df[selected_features]
    y = df['total_load(kW)']
    
    # Initialize array to store heating load predictions
    heating_load = np.zeros(len(df))
    
    # Train models and predict for each month
    for month in range(1, 13):
        month_mask = df['month'] == month
        X_month = X[month_mask]
        y_month = y[month_mask]
        
        # Only proceed if there's data for this month and if it's a heating season month
        if len(X_month) > 0 and month in [1, 2, 3, 10, 11, 12]:
            # Filter data for heating conditions
            heating_mask = df.loc[month_mask, 'air_temperature_at_2m(deg_C)'] < 18
            X_heating = X_month[heating_mask]
            y_heating = y_month[heating_mask]
            
            if len(X_heating) > 0:
                # Split data into train and test sets
                X_train, X_test, y_train, y_test = train_test_split(X_heating, y_heating, test_size=0.2, random_state=42)
                
                # Initialize and train XGBoost model
                model = xgb.XGBRegressor(random_state=42)
                model.fit(X_train, y_train)
                
                # Predict heating load for this month
                month_predictions = model.predict(X_month)
                
                # Apply heating conditions
                heating_load[month_mask] = np.where(heating_mask, month_predictions, 0)
    
    # Ensure heating load is not negative and not greater than total load
    heating_load = np.clip(heating_load, 0, df['total_load(kW)'])
    
    # Prepare output
    output_df = pd.DataFrame({
        'timestamp': df['timestamp'],
        'Heating_Load(kW)': heating_load
    })
    
    # Save output
    output_filename = filename.replace('merged_', '').replace('_cleaned', '')
    output_path = os.path.join(output_dir, output_filename)
    output_df.to_csv(output_path, index=False)
    
    print(f"Output saved to {output_path}")
    
    # Create and save visualizations
    create_visualizations(df, output_df, output_dir, output_filename)
    print(f"Visualizations saved to {output_dir}")

def create_visualizations(input_df, output_df, output_dir, filename):
    # Merge input and output dataframes
    merged_df = pd.merge(input_df, output_df, on='timestamp')
    
    # Time series plot
    plt.figure(figsize=(15, 6))
    plt.plot(merged_df['timestamp'], merged_df['total_load(kW)'], label='Total Load')
    plt.plot(merged_df['timestamp'], merged_df['Heating_Load(kW)'], label='Heating Load')
    plt.title(f'Heating Load Disaggregation Results for {filename}')
    plt.xlabel('Timestamp')
    plt.ylabel('Load (kW)')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{filename}_timeseries.png'))
    plt.close()

    # Scatter plot
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(merged_df['air_temperature_at_2m(deg_C)'], 
                          merged_df['Heating_Load(kW)'],
                          c=merged_df['timestamp'].dt.month, 
                          cmap='viridis', 
                          alpha=0.5)
    plt.colorbar(scatter, label='Month')
    plt.axvline(x=18, color='r', linestyle='--', label='Heating Threshold (18°C)')
    plt.title(f'Heating Load vs Air Temperature for {filename}')
    plt.xlabel('Air Temperature (°C)')
    plt.ylabel('Heating Load (kW)')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{filename}_scatter.png'))
    plt.close()

# Main execution
if __name__ == "__main__":
    input_dir = 'explore/05_cleaned/'
    rf_results_path = 'explore/06_analysis/random-forest-results.csv'
    output_dir = 'explore/07_results/xgboost_heating/'
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Process all files in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith('.csv'):
            input_file = os.path.join(input_dir, filename)
            print(f"Processing {filename}")
            xgboost_heating_load_disaggregation(input_file, rf_results_path, output_dir)

Processing merged_L14.B04_1H_cleaned.csv
Output saved to explore/07_results/xgboost_heating/L14.B04_1H.csv
Visualizations saved to explore/07_results/xgboost_heating/
Processing merged_L14.B03_1H_cleaned.csv
Output saved to explore/07_results/xgboost_heating/L14.B03_1H.csv
Visualizations saved to explore/07_results/xgboost_heating/
Processing merged_L14.B02_1H_cleaned.csv
Output saved to explore/07_results/xgboost_heating/L14.B02_1H.csv
Visualizations saved to explore/07_results/xgboost_heating/
Processing merged_L10.B01_30min_cleaned.csv
Output saved to explore/07_results/xgboost_heating/L10.B01_30min.csv
Visualizations saved to explore/07_results/xgboost_heating/
Processing merged_L14.B05_1H_cleaned.csv
Output saved to explore/07_results/xgboost_heating/L14.B05_1H.csv
Visualizations saved to explore/07_results/xgboost_heating/
Processing merged_L14.B01_1H_cleaned.csv
Output saved to explore/07_results/xgboost_heating/L14.B01_1H.csv
Visualizations saved to explore/07_results/xgboost_h